# Analyse croisee -- Memoire SF / Osteopathie / Syndrome de Lacomme

**Problematique** : En quoi la representation de l'osteopathie par les sages-femmes influence-t-elle la collaboration interprofessionnelle et la place qu'elles leur accordent dans la prise en charge du syndrome de Lacomme ?

## Methodologie statistique

| Croisement | Test | Justification |
|---|---|---|
| Ordinale x Ordinale (echelles 1-5) | **Spearman** | Correlation monotone entre rangs |
| Nominale x Nominale | **Chi-deux** + V de Cramer | Independance entre categories |
| Nominale (2 grp) x Ordinale | **Mann-Whitney U** | Comparaison de 2 distributions |
| Nominale (3+ grp) x Ordinale | **Kruskal-Wallis H** | Comparaison de k distributions |

Seuil : alpha = 0.05

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, mannwhitneyu, kruskal, spearmanr
from IPython.display import display, Markdown
import textwrap, warnings
warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid", font_scale=0.9)
plt.rcParams.update({'figure.dpi': 120, 'font.family': 'DejaVu Sans'})
COLORS = ['#1976D2','#FF8F00','#388E3C','#D32F2F','#7B1FA2','#00838F','#E64A19','#455A64']
print("OK")

## 1. Chargement et preparation des donnees

In [ ]:
df_raw = pd.read_excel('GRAPHIQUE_ACTUEL.xlsx', sheet_name='Feuil1', header=0)
print(f"Dimensions : {df_raw.shape[0]} repondants, {df_raw.shape[1]} variables")

In [ ]:
# Renommage simplifie
rn = {}
for i, name in enumerate(['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10','Q11','Q12',
                           'Q13','Q14','Q15_1','Q15_2','Q15_3','Q15_4','Q15_5','Q15_6',
                           'Q16','Q17','Q18','Q19','Q20','Q21','Q22','Q23','Q24','Q25',
                           'Q26','Q26_texte','Q27','Q27_texte','Q28','Q28_texte']):
    if i < len(df_raw.columns):
        rn[df_raw.columns[i]] = name
df = df_raw.rename(columns=rn)

# Nettoyage
for col in ['Q2','Q3','Q9','Q12','Q14','Q17','Q22']:
    if col in df.columns:
        df[col] = df[col].replace(0, np.nan).replace('0', np.nan)

# Collaboration binaire
df['Q8_bin'] = df['Q8'].apply(lambda x: 'Non' if x == 'Non' else ('Oui' if pd.notna(x) else np.nan))

# Collaboration ordonnee
def map_collab(val):
    s = str(val).lower()
    if val == 'Non' or s == 'non': return 0
    if 'rarement' in s: return 1
    if 'occasionnellement' in s: return 2
    if 'guli' in s: return 3
    return np.nan
df['Q8_ord'] = df['Q8'].apply(map_collab)

# Rang osteopathie dans Q15
def get_osteo_rank(row):
    for i in range(1, 7):
        c = f'Q15_{i}'
        if c in row.index and pd.notna(row[c]) and 'ost' in str(row[c]).lower():
            return i
    return np.nan
df['Q15_osteo_rank'] = df.apply(get_osteo_rank, axis=1)

print("Nettoyage OK. Colonnes :", list(df.columns[:25]))

## 2. Classification des variables

In [ ]:
ORDINALES = {'Q4','Q5','Q6','Q10','Q16','Q18','Q8_ord','Q15_osteo_rank'}
NOMINALES = {'Q1','Q3','Q7','Q13','Q17','Q22','Q23','Q24','Q8_bin'}
NOM_ORD = {'Q2','Q8','Q12'}

LABELS = {
    'Q1':'Statut','Q2':'Anciennete','Q3':'Lieu exercice',
    'Q4':'Connaissances osteo (1-5)','Q5':'Utilite osteo (1-5)',
    'Q6':'Sentiment info (1-5)','Q7':'Acces reseau','Q8':'Collaboration (detail)',
    'Q8_bin':'Collaboration (Oui/Non)','Q8_ord':'Freq. collaboration (0-3)',
    'Q10':'Connaissances SdL (1-5)','Q12':'Frequence SdL',
    'Q13':'Retard diagnostic','Q15_osteo_rank':'Rang osteo (1-6)',
    'Q16':'Favorabilite (1-5)','Q17':'Moment orientation',
    'Q18':'Info patientes (1-5)','Q22':'Influence reco officielles',
    'Q23':'Amelioration PEC','Q24':'Integration pluridisciplinaire',
}

info = []
for k in sorted(LABELS.keys()):
    t = 'Ordinale' if k in ORDINALES else 'Nominale' if k in NOMINALES else 'Nominale ordonnee'
    info.append({'Variable':k, 'Label':LABELS[k], 'Type':t})
display(pd.DataFrame(info))

## 3. Description de l'echantillon

In [ ]:
print(f"n = {len(df)}")
print()
for col, lab in [('Q1','Statut'),('Q3','Lieu exercice'),('Q7','Acces reseau'),
                  ('Q13','Retard diagnostic'),('Q23','Amelioration PEC'),('Q24','Integration')]:
    print(f"--- {lab} ({col}) ---")
    vc = df[col].value_counts()
    for val, cnt in vc.items():
        print(f"  {val} : {cnt} ({cnt/len(df)*100:.1f}%)")
    print()

print("--- Variables ordinales (1-5) ---")
rows = []
for col, lab in [('Q4','Conn. osteo'),('Q5','Utilite'),('Q6','Info SF'),
                  ('Q10','Conn. SdL'),('Q16','Favorabilite'),('Q18','Info patientes')]:
    s = df[col].describe()
    rows.append({'Variable':lab,'n':int(s['count']),'Moy':f"{s['mean']:.2f}",
                 'ET':f"{s['std']:.2f}",'Med':f"{s['50%']:.0f}"})
display(pd.DataFrame(rows).set_index('Variable'))

## 4. Fonctions d'analyse

**Logique de selection du test** : le test est determine par la nature des deux variables croisees.
Chaque analyse affiche systematiquement le tableau de contingence, le test statistique, et les graphiques detailles.

In [ ]:
def get_type(v):
    if v in ORDINALES: return 'ord'
    if v in NOMINALES: return 'nom'
    if v in NOM_ORD: return 'nom_ord'
    return 'autre'

def select_test(v1, v2, data):
    t1, t2 = get_type(v1), get_type(v2)
    if t1 == 'ord' and t2 == 'ord':
        return 'spearman', f'{v1}(ord) x {v2}(ord) -> Spearman'
    if t1 in ('nom','nom_ord') and t2 in ('nom','nom_ord'):
        return 'chi2', f'{v1}(nom) x {v2}(nom) -> Chi-deux'
    # une nominale x une ordinale
    if t1 in ('nom','nom_ord') and t2 == 'ord':
        nom, ordn = v1, v2
    elif t2 in ('nom','nom_ord') and t1 == 'ord':
        nom, ordn = v2, v1
    else:
        return 'chi2', f'{v1}({t1}) x {v2}({t2}) -> Chi-deux (defaut)'
    ng = data[nom].nunique()
    if ng == 2:
        return 'mw', f'{nom}(nom,2grp) x {ordn}(ord) -> Mann-Whitney U'
    return 'kw', f'{nom}(nom,{ng}grp) x {ordn}(ord) -> Kruskal-Wallis H'

def interp_p(p):
    if p is None: return 'N/A'
    if p < 0.001: return '*** (p<0.001)'
    if p < 0.01: return '** (p<0.01)'
    if p < 0.05: return '* (p<0.05)'
    return f'NS (p={p:.3f})'

def interp_v(v):
    if v < 0.1: return 'negligeable'
    if v < 0.3: return 'faible'
    if v < 0.5: return 'modere'
    return 'fort'

def interp_rho(r):
    r = abs(r)
    if r < 0.1: return 'negligeable'
    if r < 0.3: return 'faible'
    if r < 0.5: return 'moderee'
    if r < 0.7: return 'forte'
    return 'tres forte'

def wl(labels, w=18):
    return ['\n'.join(textwrap.wrap(str(l), w)) for l in labels]

print("Fonctions OK")

In [ ]:
def analyse(v1, v2, titre, question=""):
    data = df[[v1, v2]].copy().replace(0, np.nan).dropna()
    n = len(data)
    if n < 10:
        print(f"  Donnees insuffisantes (n={n})")
        return None
    l1, l2 = LABELS.get(v1, v1), LABELS.get(v2, v2)
    test, just = select_test(v1, v2, data)
    
    display(Markdown(f"### {titre}"))
    if question:
        display(Markdown(f"*{question}*"))
    display(Markdown(f"**{l1}** x **{l2}** | n = {n} | Test : {just}"))
    
    res = {'titre':titre,'var1':v1,'var2':v2,'l1':l1,'l2':l2,'n':n,'test':test}
    
    # ========== CHI-DEUX ==========
    if test == 'chi2':
        ct = pd.crosstab(data[v1], data[v2])
        ct_m = pd.crosstab(data[v1], data[v2], margins=True, margins_name='Total')
        ct_pctc = pd.crosstab(data[v1], data[v2], normalize='columns') * 100
        ct_pctr = pd.crosstab(data[v1], data[v2], normalize='index') * 100
        
        if ct.shape[0] < 2 or ct.shape[1] < 2:
            print("  Tableau trop petit")
            return None
        
        chi2, p, dof, exp = chi2_contingency(ct)
        N = ct.sum().sum()
        V = np.sqrt(chi2 / (N * (min(ct.shape)-1)))
        exp_df = pd.DataFrame(exp, index=ct.index, columns=ct.columns).round(1)
        pct_low = (exp < 5).sum() / exp.size * 100
        residus = (ct.values - exp) / np.sqrt(exp)
        res_df = pd.DataFrame(residus, index=ct.index, columns=ct.columns).round(2)
        
        display(Markdown("**Effectifs observes :**"))
        display(ct_m)
        display(Markdown("**Pourcentages en colonne (%) :**"))
        display(ct_pctc.round(1))
        display(Markdown("**Pourcentages en ligne (%) :**"))
        display(ct_pctr.round(1))
        display(Markdown("**Effectifs theoriques :**"))
        display(exp_df)
        if pct_low > 20:
            display(Markdown(f"**Attention : {pct_low:.0f}% des effectifs theoriques < 5.**"))
        display(Markdown("**Residus standardises (contributions au Chi2) :**"))
        display(res_df)
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))
        fig.suptitle(titre + f'  (n={n})', fontsize=12, fontweight='bold', y=1.02)
        
        sns.heatmap(ct, annot=True, fmt='d', cmap='Blues', ax=axes[0], linewidths=0.8)
        axes[0].set_title('Effectifs observes', fontweight='bold')
        axes[0].set_xticklabels(wl(ct.columns,14), rotation=45, ha='right', fontsize=7)
        axes[0].set_yticklabels(wl(ct.index,18), rotation=0, fontsize=7)
        
        ct_pctc_plot = ct.div(ct.sum(axis=0), axis=1) * 100
        ct_pctc_plot.T.plot(kind='bar', stacked=True, ax=axes[1],
                           color=COLORS[:len(ct.index)], edgecolor='white', linewidth=0.5)
        axes[1].set_title('% en colonne (empile)', fontweight='bold')
        axes[1].set_xticklabels(wl(ct.columns,14), rotation=45, ha='right', fontsize=7)
        axes[1].legend(title='', bbox_to_anchor=(1.02,1), loc='upper left', fontsize=6)
        axes[1].set_ylim(0,110)
        axes[1].set_ylabel('%')
        
        sns.heatmap(res_df, annot=True, fmt='.2f', cmap='RdBu_r', center=0, ax=axes[2],
                   linewidths=0.8, vmin=-3, vmax=3)
        axes[2].set_title('Residus standardises', fontweight='bold')
        axes[2].set_xticklabels(wl(ct.columns,14), rotation=45, ha='right', fontsize=7)
        axes[2].set_yticklabels(wl(ct.index,18), rotation=0, fontsize=7)
        
        fig.text(0.5, -0.02,
                f'Chi2={chi2:.3f}  ddl={dof}  p={p:.4f} {interp_p(p)}  |  V de Cramer={V:.3f} ({interp_v(V)})',
                ha='center', fontsize=9, bbox=dict(facecolor='lightyellow',alpha=0.8,boxstyle='round'))
        plt.tight_layout()
        plt.show()
        
        res.update({'chi2':chi2,'p':p,'dof':dof,'V':V,'pct_low':pct_low})
    
    # ========== MANN-WHITNEY ==========
    elif test == 'mw':
        t1 = get_type(v1)
        if t1 in ('nom','nom_ord'): cat, ordn, clab, olab = v1, v2, l1, l2
        else: cat, ordn, clab, olab = v2, v1, l2, l1
        
        grps = {name: g[ordn].values for name, g in data.groupby(cat) if len(g) >= 3}
        gnames = list(grps.keys())
        if len(gnames) < 2:
            print("  Moins de 2 groupes")
            return None
        g1, g2 = grps[gnames[0]], grps[gnames[1]]
        
        ct_m = pd.crosstab(data[cat], data[ordn], margins=True, margins_name='Total')
        ct_pctr = pd.crosstab(data[cat], data[ordn], normalize='index') * 100
        display(Markdown("**Effectifs observes :**"))
        display(ct_m)
        display(Markdown("**Pourcentages en ligne (%) :**"))
        display(ct_pctr.round(1))
        display(Markdown("**Statistiques par groupe :**"))
        display(data.groupby(cat)[ordn].describe().round(2))
        
        U, p = mannwhitneyu(g1, g2, alternative='two-sided')
        n1, n2 = len(g1), len(g2)
        mu_U = n1*n2/2
        sig_U = np.sqrt(n1*n2*(n1+n2+1)/12)
        Z = (U - mu_U) / sig_U
        r_eff = abs(Z) / np.sqrt(n1+n2)
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))
        fig.suptitle(titre + f'  (n={n})', fontsize=12, fontweight='bold', y=1.02)
        
        order = sorted(data[cat].unique(), key=str)
        sns.boxplot(data=data, x=cat, y=ordn, ax=axes[0], palette=COLORS, order=order, width=0.5)
        sns.stripplot(data=data, x=cat, y=ordn, ax=axes[0], color='black', alpha=0.3, size=3, jitter=True, order=order)
        meds = data.groupby(cat)[ordn].median()
        mns = data.groupby(cat)[ordn].mean()
        for i, c in enumerate(order):
            if c in meds.index:
                axes[0].text(i, meds[c]+0.15, f'Md={meds[c]:.1f}\nM={mns[c]:.2f}',
                           ha='center', fontsize=7, fontweight='bold', color='darkred')
        axes[0].set_title('Boxplot + points individuels', fontweight='bold')
        axes[0].set_xticklabels(wl(order,18), fontsize=7)
        
        sns.violinplot(data=data, x=cat, y=ordn, ax=axes[1], palette=COLORS, order=order, inner='quartile')
        axes[1].set_title('Violinplot (distribution)', fontweight='bold')
        axes[1].set_xticklabels(wl(order,18), fontsize=7)
        
        ct_no = pd.crosstab(data[ordn], data[cat])
        ct_no_pct = ct_no.div(ct_no.sum(axis=0), axis=1) * 100
        ct_no_pct.plot(kind='bar', ax=axes[2], color=COLORS[:len(order)], edgecolor='white')
        axes[2].set_title('Distribution des scores par groupe (%)', fontweight='bold')
        axes[2].set_xlabel(olab)
        axes[2].set_ylabel('%')
        axes[2].legend(title=clab, fontsize=7)
        
        r_int = 'negligeable' if r_eff<0.1 else 'faible' if r_eff<0.3 else 'modere' if r_eff<0.5 else 'fort'
        fig.text(0.5, -0.02,
                f'U={U:.1f}  Z={Z:.3f}  p={p:.4f} {interp_p(p)}  |  r={r_eff:.3f} ({r_int})',
                ha='center', fontsize=9, bbox=dict(facecolor='lightyellow',alpha=0.8,boxstyle='round'))
        plt.tight_layout()
        plt.show()
        
        res.update({'U':U,'p':p,'Z':Z,'r':r_eff})
    
    # ========== KRUSKAL-WALLIS ==========
    elif test == 'kw':
        t1 = get_type(v1)
        if t1 in ('nom','nom_ord'): cat, ordn, clab, olab = v1, v2, l1, l2
        else: cat, ordn, clab, olab = v2, v1, l2, l1
        
        grps = {name: g[ordn].values for name, g in data.groupby(cat) if len(g) >= 3}
        gnames = list(grps.keys())
        if len(gnames) < 2:
            print("  Moins de 2 groupes")
            return None
        
        ct_m = pd.crosstab(data[cat], data[ordn], margins=True, margins_name='Total')
        ct_pctr = pd.crosstab(data[cat], data[ordn], normalize='index') * 100
        display(Markdown("**Effectifs observes :**"))
        display(ct_m)
        display(Markdown("**Pourcentages en ligne (%) :**"))
        display(ct_pctr.round(1))
        display(Markdown("**Statistiques par groupe :**"))
        display(data.groupby(cat)[ordn].describe().round(2))
        
        arrays = [grps[k] for k in gnames]
        H, p = kruskal(*arrays)
        N = sum(len(a) for a in arrays)
        k_g = len(arrays)
        eta2 = (H - k_g + 1) / (N - k_g) if N > k_g else 0
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))
        fig.suptitle(titre + f'  (n={n})', fontsize=12, fontweight='bold', y=1.02)
        
        order = sorted(data[cat].dropna().unique(), key=str)
        sns.boxplot(data=data, x=cat, y=ordn, ax=axes[0], palette=COLORS, order=order, width=0.5)
        sns.stripplot(data=data, x=cat, y=ordn, ax=axes[0], color='black', alpha=0.3, size=3, jitter=True, order=order)
        meds = data.groupby(cat)[ordn].median()
        mns = data.groupby(cat)[ordn].mean()
        for i, c in enumerate(order):
            if c in meds.index:
                axes[0].text(i, meds[c]+0.15, f'Md={meds[c]:.1f}\nM={mns[c]:.2f}',
                           ha='center', fontsize=6, fontweight='bold', color='darkred')
        axes[0].set_title('Boxplot + points individuels', fontweight='bold')
        axes[0].set_xticklabels(wl(order,14), rotation=45, ha='right', fontsize=7)
        
        # Moyennes + IC95
        ms = data.groupby(cat)[ordn].agg(['mean','std','count']).reindex(order)
        ms['se'] = ms['std'] / np.sqrt(ms['count'])
        ms['ci'] = ms['se'] * 1.96
        axes[1].bar(range(len(ms)), ms['mean'], yerr=ms['ci'], capsize=4,
                   color=COLORS[:len(ms)], edgecolor='white')
        axes[1].set_xticks(range(len(ms)))
        axes[1].set_xticklabels(wl(order,14), rotation=45, ha='right', fontsize=7)
        axes[1].set_title('Moyenne +/- IC 95%', fontweight='bold')
        axes[1].set_ylabel(olab)
        for i, (idx, row) in enumerate(ms.iterrows()):
            axes[1].text(i, row['mean']+row['ci']+0.05, f'{row["mean"]:.2f}\n(n={int(row["count"])})',
                        ha='center', fontsize=7)
        
        ct_heat = pd.crosstab(data[cat], data[ordn], normalize='index') * 100
        sns.heatmap(ct_heat, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[2], linewidths=0.5)
        axes[2].set_title('Repartition (% en ligne)', fontweight='bold')
        axes[2].set_yticklabels(wl(ct_heat.index,14), rotation=0, fontsize=7)
        
        eta_int = 'negligeable' if eta2<0.01 else 'faible' if eta2<0.06 else 'modere' if eta2<0.14 else 'fort'
        fig.text(0.5, -0.02,
                f'H={H:.3f}  ddl={k_g-1}  p={p:.4f} {interp_p(p)}  |  eta2={eta2:.3f} ({eta_int})',
                ha='center', fontsize=9, bbox=dict(facecolor='lightyellow',alpha=0.8,boxstyle='round'))
        plt.tight_layout()
        plt.show()
        
        # Post-hoc si significatif
        if p < 0.05 and k_g > 2:
            display(Markdown("**Comparaisons post-hoc (Bonferroni) :**"))
            ph = []
            nc = k_g*(k_g-1)/2
            for i in range(len(gnames)):
                for j in range(i+1, len(gnames)):
                    u_ph, p_ph = mannwhitneyu(grps[gnames[i]], grps[gnames[j]], alternative='two-sided')
                    p_corr = min(p_ph * nc, 1.0)
                    ph.append({'Comp':f'{gnames[i]} vs {gnames[j]}', 'U':f'{u_ph:.1f}',
                              'p brut':f'{p_ph:.4f}', 'p Bonf.':f'{p_corr:.4f}',
                              'Sig.':'Oui' if p_corr<0.05 else 'Non'})
            display(pd.DataFrame(ph))
        
        res.update({'H':H,'p':p,'eta2':eta2,'k':k_g})
    
    # ========== SPEARMAN ==========
    elif test == 'spearman':
        ct = pd.crosstab(data[v1], data[v2])
        ct_m = pd.crosstab(data[v1], data[v2], margins=True, margins_name='Total')
        ct_pct = pd.crosstab(data[v1], data[v2], normalize='all') * 100
        
        display(Markdown("**Effectifs croises :**"))
        display(ct_m)
        display(Markdown("**Pourcentages du total (%) :**"))
        display(ct_pct.round(1))
        
        rho, p = spearmanr(data[v1], data[v2])
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))
        fig.suptitle(titre + f'  (n={n})', fontsize=12, fontweight='bold', y=1.02)
        
        jit1 = data[v1].astype(float) + np.random.normal(0, 0.12, n)
        jit2 = data[v2].astype(float) + np.random.normal(0, 0.12, n)
        axes[0].scatter(jit1, jit2, alpha=0.4, color=COLORS[0], edgecolors='white', s=30)
        z = np.polyfit(data[v1].astype(float), data[v2].astype(float), 1)
        xr = np.linspace(data[v1].min(), data[v1].max(), 100)
        axes[0].plot(xr, np.poly1d(z)(xr), 'r--', lw=2, label=f'rho={rho:.3f}')
        axes[0].set_xlabel(l1); axes[0].set_ylabel(l2)
        axes[0].set_title('Nuage de points + tendance', fontweight='bold')
        axes[0].legend(fontsize=8)
        
        sns.heatmap(ct, annot=True, fmt='d', cmap='Blues', ax=axes[1], linewidths=0.5)
        axes[1].set_title('Effectifs croises', fontweight='bold')
        
        mv = data.groupby(v1)[v2].agg(['mean','std','count'])
        axes[2].bar(mv.index.astype(int), mv['mean'], yerr=mv['std'], capsize=4,
                   color=COLORS[0], edgecolor='white', alpha=0.8)
        axes[2].set_xlabel(l1)
        axes[2].set_ylabel(f'Moyenne {l2}')
        axes[2].set_title(f'Moyenne de {l2} par niveau de {l1}', fontweight='bold')
        for x, row in mv.iterrows():
            axes[2].text(x, row['mean']+row['std']+0.05, f'{row["mean"]:.2f}\n(n={int(row["count"])})',
                        ha='center', fontsize=7)
        
        fig.text(0.5, -0.02,
                f'Spearman rho={rho:.3f}  p={p:.4f} {interp_p(p)}  |  Force : {interp_rho(rho)}',
                ha='center', fontsize=9, bbox=dict(facecolor='lightyellow',alpha=0.8,boxstyle='round'))
        plt.tight_layout()
        plt.show()
        
        res.update({'rho':rho,'p':p})
    
    print("---")
    return res

print("Fonction analyse() prete.")


---
## AXE 1 : REPRESENTATIONS DE L'OSTEOPATHIE
### 1.1 Connaissances comme fondement

In [ ]:
results = []

In [ ]:
r = analyse('Q4','Q1', "Axe 1.1 -- Connaissances osteo selon statut", "Les connaissances different-elles entre etudiants et diplomes ?")
if r: results.append(r)

In [ ]:
r = analyse('Q4','Q2', "Axe 1.1 -- Connaissances osteo selon anciennete", "Les SF experimentees connaissent-elles mieux l'osteopathie ?")
if r: results.append(r)

In [ ]:
r = analyse('Q6','Q4', "Axe 1.1 -- Coherence sentiment info / connaissances", "Coherence entre connaissance objective et subjective ?")
if r: results.append(r)

### 1.2 Perception utilite et efficacite

In [ ]:
r = analyse('Q5','Q4', "Axe 1.2 -- Utilite percue selon connaissances", "Plus on connait, plus on percoit l'utilite ?")
if r: results.append(r)

In [ ]:
r = analyse('Q5','Q16', "Axe 1.2 -- Coherence utilite / favorabilite", "Utilite et favorabilite sont-elles coherentes ?")
if r: results.append(r)

In [ ]:
r = analyse('Q16','Q1', "Axe 1.2 -- Favorabilite selon statut", "Le statut influence-t-il la favorabilite ?")
if r: results.append(r)

In [ ]:
r = analyse('Q17','Q16', "Axe 1.2 -- Moment orientation selon favorabilite", "Les plus favorables orientent-elles plus tot ?")
if r: results.append(r)

In [ ]:
r = analyse('Q15_osteo_rank','Q3', "Axe 1.2 -- Classement osteo selon lieu exercice", "Priorites differentes liberal/hopital ?")
if r: results.append(r)

### 1.3 Facteurs influencant les representations

In [ ]:
r = analyse('Q16','Q2', "Axe 1.3 -- Favorabilite selon anciennete", "L'anciennete influence-t-elle la favorabilite ?")
if r: results.append(r)

In [ ]:
r = analyse('Q4','Q3', "Axe 1.3 -- Connaissances selon lieu exercice", "Le lieu influence-t-il les connaissances ?")
if r: results.append(r)

---
## AXE 2 : PLACE ACCORDEE A L'OSTEOPATHIE
### 2.1 Place dans la PEC du SdL

In [ ]:
r = analyse('Q17','Q12', "Axe 2.1 -- Orientation selon frequence SdL", "Plus on voit le SdL, mieux on oriente ?")
if r: results.append(r)

In [ ]:
r = analyse('Q17','Q8_bin', "Axe 2.1 -- Orientation selon collaboration", "La collaboration favorise-t-elle orientation precoce ?")
if r: results.append(r)

### 2.2 Place dans la pratique globale

In [ ]:
r = analyse('Q8','Q3', "Axe 2.2 -- Collaboration selon lieu exercice", "Qui collabore le plus ?")
if r: results.append(r)

In [ ]:
r = analyse('Q8_ord','Q2', "Axe 2.2 -- Freq. collaboration selon anciennete", "L'anciennete influence-t-elle la collaboration ?")
if r: results.append(r)

### 2.3 Freins limitant la place

In [ ]:
r = analyse('Q7','Q8_bin', "Axe 2.3 -- Reseau et collaboration", "L'absence de reseau est-elle LE frein ?")
if r: results.append(r)

In [ ]:
r = analyse('Q18','Q6', "Axe 2.3 -- Info patientes vs info SF", "SF mal informees = patientes mal informees ?")
if r: results.append(r)

In [ ]:
r = analyse('Q13','Q3', "Axe 2.3 -- Retard diagnostic selon lieu", "Le retard est-il percu differemment ?")
if r: results.append(r)

---
## AXE 3 : INFLUENCE SUR LA COLLABORATION
### 3.1 Etat de la collaboration

In [ ]:
r = analyse('Q8_bin','Q16', "Axe 3.1 -- Collaboration selon favorabilite", "Les favorables collaborent-elles plus ?")
if r: results.append(r)

In [ ]:
r = analyse('Q8_bin','Q5', "Axe 3.1 -- Collaboration selon utilite percue", "Percevoir l'utilite -> collaboration ?")
if r: results.append(r)

In [ ]:
r = analyse('Q7','Q8_bin', "Axe 3.1 -- Collaboration selon reseau", "Le reseau est-il determinant ?")
if r: results.append(r)

In [ ]:
r = analyse('Q8_bin','Q4', "Axe 3.1 -- Collaboration selon connaissances", "Mieux connaitre -> plus collaborer ?")
if r: results.append(r)

### 3.2 Ecart intention/comportement

In [ ]:
r = analyse('Q16','Q8_ord', "Axe 3.2 -- Favorabilite vs freq. collaboration", "La favorabilite se traduit-elle en actes ?")
if r: results.append(r)

### 3.3 Role du cadre institutionnel (H3)

In [ ]:
r = analyse('Q22','Q8_bin', "Axe 3.3 -- Reco officielles et collaboration", "Celles influencees collaborent-elles plus ?")
if r: results.append(r)

In [ ]:
r = analyse('Q22','Q2', "Axe 3.3 -- Reco selon anciennete", "Jeunes diplomees plus sensibles aux reco ?")
if r: results.append(r)

In [ ]:
r = analyse('Q22','Q1', "Axe 3.3 -- Reco selon statut", "Etudiantes vs diplomees face aux reco")
if r: results.append(r)

### 3.4 Effet feedback

In [ ]:
r = analyse('Q8_ord','Q5', "Axe 3.4 -- Collaboration renforce utilite percue ?", "Cercle vertueux collaboration -> representations ?")
if r: results.append(r)

---
## AXE 4 : LE SYNDROME DE LACOMME
### 4.1 Connaissances SdL

In [ ]:
r = analyse('Q10','Q4', "Axe 4.1 -- Connaissances SdL vs osteopathie", "Connait-on mieux le syndrome ou le traitement ?")
if r: results.append(r)

In [ ]:
r = analyse('Q10','Q12', "Axe 4.1 -- Connaissances SdL selon frequence", "Rencontrer souvent ameliore les connaissances ?")
if r: results.append(r)

In [ ]:
r = analyse('Q10','Q1', "Axe 4.1 -- Connaissances SdL selon statut", "Le referentiel s'est-il ameliore ?")
if r: results.append(r)

### 4.2 Retard diagnostic (H4)

In [ ]:
r = analyse('Q13','Q10', "Axe 4.2 -- Retard selon connaissances SdL", "Celles qui connaissent mieux percoivent plus le retard ?")
if r: results.append(r)

In [ ]:
r = analyse('Q13','Q2', "Axe 4.2 -- Retard selon anciennete", "Causes differentes selon experience ?")
if r: results.append(r)

In [ ]:
r = analyse('Q13','Q17', "Axe 4.2 -- Retard et moment orientation", "Percevoir retard influence orientation ?")
if r: results.append(r)

### 4.3 SdL comme cas d'ecole

In [ ]:
r = analyse('Q15_osteo_rank','Q16', "Axe 4.3 -- Rang osteo selon favorabilite", "Les favorables classent-elles mieux l'osteo ?")
if r: results.append(r)

In [ ]:
r = analyse('Q15_osteo_rank','Q10', "Axe 4.3 -- Rang osteo selon connaissances SdL", "Connaissances SdL influencent choix therapeutiques ?")
if r: results.append(r)

---
## AXE 5 : PERSPECTIVES
### 5.1 Volonte d'evolution

In [ ]:
r = analyse('Q23','Q8_bin', "Axe 5.1 -- Amelioration PEC selon collaboration", "Non-collaborantes reconnaissent l'interet ?")
if r: results.append(r)

In [ ]:
r = analyse('Q24','Q8_bin', "Axe 5.1 -- Integration selon collaboration", "Non-collaborantes favorables a integration ?")
if r: results.append(r)

In [ ]:
r = analyse('Q23','Q24', "Axe 5.1 -- Coherence amelioration / integration", "Coherence entre les deux dimensions ?")
if r: results.append(r)

### 5.2 Conditions necessaires

In [ ]:
r = analyse('Q24','Q2', "Axe 5.2 -- Integration selon anciennete", "Priorites selon experience ?")
if r: results.append(r)

In [ ]:
r = analyse('Q24','Q3', "Axe 5.2 -- Integration selon lieu", "Memes priorites liberal/hopital ?")
if r: results.append(r)

In [ ]:
r = analyse('Q23','Q1', "Axe 5.2 -- Amelioration PEC selon statut", "Vision differente etudiants/diplomes ?")
if r: results.append(r)

---
## 6. Matrice de correlation (Spearman)

In [ ]:
nvars = ['Q4','Q5','Q6','Q10','Q16','Q18','Q8_ord','Q15_osteo_rank']
nlabs = ['Conn.\nosteo','Utilite','Info\nSF','Conn.\nSdL','Favorab.','Info\npat.','Collab.','Rang\nosteo']
ex = [v for v in nvars if v in df.columns]
el = [nlabs[nvars.index(v)] for v in ex]
cd = df[ex].dropna()
print(f"Matrice sur n = {len(cd)} observations")

nv = len(ex)
rho_m = np.zeros((nv,nv))
p_m = np.zeros((nv,nv))
for i in range(nv):
    for j in range(nv):
        if i==j: rho_m[i,j]=1.0
        else:
            r,p = spearmanr(cd[ex[i]], cd[ex[j]])
            rho_m[i,j]=r; p_m[i,j]=p

ann = []
for i in range(nv):
    row = []
    for j in range(nv):
        s = ''
        if i!=j:
            if p_m[i,j]<0.001: s='***'
            elif p_m[i,j]<0.01: s='**'
            elif p_m[i,j]<0.05: s='*'
        row.append(f'{rho_m[i,j]:.2f}{s}')
    ann.append(row)

rdf = pd.DataFrame(rho_m, index=el, columns=el)
adf = pd.DataFrame(ann, index=el, columns=el)
mask = np.triu(np.ones_like(rho_m, dtype=bool))

fig, ax = plt.subplots(figsize=(10,8))
sns.heatmap(rdf, mask=mask, annot=adf, fmt='', cmap='RdBu_r', center=0, vmin=-1, vmax=1,
           square=True, linewidths=1, ax=ax, cbar_kws={'shrink':0.8})
ax.set_title('Matrice de correlation (Spearman)\n* p<0.05  ** p<0.01  *** p<0.001',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelations significatives :")
sc = []
for i in range(nv):
    for j in range(i+1,nv):
        if p_m[i,j] < 0.05:
            sc.append({'Var1':el[i].replace('\n',' '),'Var2':el[j].replace('\n',' '),
                       'rho':round(rho_m[i,j],3),'p':round(p_m[i,j],4),'Force':interp_rho(rho_m[i,j])})
display(pd.DataFrame(sc))

---
## 7. Analyse de profils

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 11))
fig.suptitle('Synthese des profils', fontsize=14, fontweight='bold')

ct1 = pd.crosstab(df['Q1'], df['Q16']>=4)
ct1.columns = ['Score < 4','Score >= 4']
ct1p = ct1.div(ct1.sum(axis=1), axis=0)*100
ct1p.plot(kind='bar', stacked=True, ax=axes[0,0], color=[COLORS[1],COLORS[2]], edgecolor='white')
axes[0,0].set_title('Favorabilite par statut', fontweight='bold')
axes[0,0].set_ylabel('%'); axes[0,0].set_xticklabels(wl(ct1.index,20), rotation=0, fontsize=7)

ct2d = df[['Q3','Q8_bin']].dropna()
ct2 = pd.crosstab(ct2d['Q3'], ct2d['Q8_bin'])
ct2p = ct2.div(ct2.sum(axis=1), axis=0)*100
ct2p.plot(kind='bar', stacked=True, ax=axes[0,1], color=[COLORS[3],COLORS[0]], edgecolor='white')
axes[0,1].set_title('Collaboration par lieu', fontweight='bold')
axes[0,1].set_ylabel('%'); axes[0,1].set_xticklabels(wl(ct2.index,15), rotation=0, fontsize=7)

ad = df[['Q2','Q4','Q10']].dropna()
if ad['Q2'].nunique()>1:
    m = ad.groupby('Q2')[['Q4','Q10']].mean()
    m.columns = ['Conn. Osteo','Conn. SdL']
    m.plot(kind='bar', ax=axes[1,0], color=[COLORS[0],COLORS[4]], edgecolor='white')
    axes[1,0].set_title('Connaissances par anciennete', fontweight='bold')
    axes[1,0].set_ylabel('Score moyen'); axes[1,0].set_xticklabels(wl(m.index,14), rotation=45, ha='right', fontsize=7)

ct3d = df[['Q7','Q8_bin']].dropna()
ct3 = pd.crosstab(ct3d['Q7'], ct3d['Q8_bin'])
ct3p = ct3.div(ct3.sum(axis=1), axis=0)*100
ct3p.plot(kind='bar', stacked=True, ax=axes[1,1], color=[COLORS[3],COLORS[2]], edgecolor='white')
axes[1,1].set_title('Collaboration selon reseau', fontweight='bold')
axes[1,1].set_ylabel('%'); axes[1,1].set_xticklabels(wl(ct3.index,15), rotation=0, fontsize=7)

plt.tight_layout()
plt.show()

---
## 8. Synthese et validation des hypotheses

In [ ]:
display(Markdown("### Tableau recapitulatif"))
rows = []
for r in results:
    if r is None: continue
    t = r['test']
    if t=='chi2': st=f"chi2={r.get('chi2',0):.3f}"; ef=f"V={r.get('V',0):.3f} ({interp_v(r.get('V',0))})"
    elif t=='spearman': st=f"rho={r.get('rho',0):.3f}"; ef=f"|rho|={abs(r.get('rho',0)):.3f} ({interp_rho(r.get('rho',0))})"
    elif t=='mw': st=f"U={r.get('U',0):.1f}"; ef=f"r={r.get('r',0):.3f}"
    elif t=='kw': st=f"H={r.get('H',0):.3f}"; ef=f"eta2={r.get('eta2',0):.3f}"
    else: st=""; ef=""
    p = r.get('p')
    rows.append({
        'Croisement':r['titre'][:55], 'n':r['n'],
        'Test':t.replace('chi2','Chi-deux').replace('mw','Mann-Whitney').replace('kw','Kruskal-Wallis').replace('spearman','Spearman'),
        'Stat':st, 'p':f'{p:.4f}' if p else 'N/A',
        'Sig':'Oui' if p and p<0.05 else 'Non', 'Effet':ef
    })
sdf = pd.DataFrame(rows)

def hl(row):
    if row['Sig']=='Oui':
        p=float(row['p'])
        if p<0.001: return ['background-color:#c8e6c9']*len(row)
        if p<0.01: return ['background-color:#dcedc8']*len(row)
        return ['background-color:#fff9c4']*len(row)
    return ['']*len(row)

display(sdf.style.apply(hl, axis=1).set_caption('Vert fonce=p<0.001 | Vert clair=p<0.01 | Jaune=p<0.05'))

tot=len(rows); sig=sum(1 for r in rows if r['Sig']=='Oui')
print(f"\nTotal : {tot} analyses | Significatifs : {sig} ({sig/tot*100:.1f}%) | NS : {tot-sig}")

In [ ]:
display(Markdown("### Validation des hypotheses"))

print("H1 : SF favorables au recours osteo")
print(f"  Q16 moyenne={df['Q16'].mean():.2f}, mediane={df['Q16'].median():.0f}")
print(f"  % score >= 4 : {(df['Q16']>=4).mean()*100:.1f}%")
print(f"  --> H1 VALIDEE")
print()

print("H2 : Facteurs individuels influencent recommandation")
for r in results:
    if r and (r['var1'] in ['Q1','Q2','Q3'] or r['var2'] in ['Q1','Q2','Q3']):
        if r['var1'] in ['Q16','Q4','Q8_bin','Q5'] or r['var2'] in ['Q16','Q4','Q8_bin','Q5']:
            p=r.get('p',1)
            print(f"  {r['l1']} x {r['l2']} : p={p:.4f} {'SIGNIFICATIF' if p<0.05 else 'NS'}")
print()

print("H3 : Absence cadre reglementaire = frein majeur")
for r in results:
    if r and ('Q22' in [r['var1'],r['var2']]):
        p=r.get('p',1)
        print(f"  {r['l1']} x {r['l2']} : p={p:.4f} {'SIGNIFICATIF' if p<0.05 else 'NS'}")
print()

print("H4 : Retard diagnostic")
print(f"  % percevant retard : {(df['Q13']=='Oui').mean()*100:.1f}%")
for r in results:
    if r and ('Q13' in [r['var1'],r['var2']]):
        p=r.get('p',1)
        print(f"  {r['l1']} x {r['l2']} : p={p:.4f} {'SIGNIFICATIF' if p<0.05 else 'NS'}")
print()

print("H5 : Osteo percue comme complementaire pertinente")
print(f"  Q5 moyenne={df['Q5'].mean():.2f}, mediane={df['Q5'].median():.0f}")
print(f"  % score >= 4 : {(df['Q5']>=4).mean()*100:.1f}%")
print(f"  --> H5 VALIDEE")